# Analysis of OrthoFinder results

Objectives:
1. Find the orthogroups that include genes EC1.1 and EC1.2 and isolate the orthologs, along with corresponding species
2. Write the FASTA amino acid sequence of these genes into a separate file
3. Find the upstream sequence

Folder structure:
* Current folder - results of OrthoFinder - `Orthogroup.tsv` from each run, renamed to `orthogroups_i.tsv`, where i corresponds to the number of the group
* Folders in current folder:
    * protein/ - folder with Amino Acid FASTA sequences
    * annotation/ - folder with annotations
    * genome/ - folder with genomes

Versions:
* OrthoFinder - v2.5.4
* Python - 3.8.8
* pandas - 1.4.1
* biopython - 1.79

In [363]:
# Python standard libraries
from functools import reduce
import itertools
import os
import re

# Python additional libraries
import pandas as pd
from Bio import SeqIO

Writing the function that takes the `Orthogroup.tsv` from the folder `Orthogroups` in the OrthoFinder Result output and saves the results. For each group, the `Orthogroup.tsv` has been renamed to `orthogroups_i.tsv`, where i corresponds to the number of the group.

In [364]:
def find_ec_orthogroup(path_to_orthogroup_tsv, path_to_output_tsv):
    """
    Takes the resulting Orthogroups.tsv file from OrthoFinder and returns the .tsv file with EC gene orthogroups
    :param path_to_orthogroup_tsv: str, path to  .tsv file
    :param path_to_output_tsv: str, path to output .tsv file
    :return: None, saves the output as .tsv
    """
    o = pd.read_csv(path_to_orthogroup_tsv, sep='\t')
    ec1_row = -1
    ec2_row = -1
    # Find the indices of EC genes 
    for i, row in enumerate(o['Arabidopsis_thaliana.TAIR10.pep.all'].str.split(",")):
        if type(row) == list:
            for gene in row:
                if gene.strip().split(".")[0] == "AT1G76750":
                    ec1_row = i
                elif gene.strip().split(".")[0] == "AT2G21740":
                    ec2_row = i
    # If both EC genes in one orthogroup
    if ec1_row == ec2_row:
        o.iloc[[ec1_row]].to_csv(path_to_output_tsv, sep="\t")
        return o.iloc[[ec1_row]]
    else:
        if ec1_row >= 0 and ec2_row >= 0:
            # Select rows with the EC genes
            o_ec = o.iloc[[ec1_row, ec2_row]]
            # Saving the orthologs as a result
            o_ec.to_csv(path_to_output_tsv, sep="\t")
            # No NANs in rows -> combine the orthogroups of two EC genes
            o_ec.loc[ec1_row, ~o_ec.isnull().any(axis=0)] = o_ec.loc[ec1_row, ~o_ec.isnull().any(axis=0)] + ", " + o_ec.loc[ec2_row, ~o_ec.isnull().any(axis=0)]
            # Select part of dataframe where there are Nans
            o_ec_nan = o_ec.loc[o_ec.isnull().any(axis=1), o_ec.isnull().any(axis=0)]       
            # Combine the Nan values into one row (Series)
            # If the Nan values are in two rows, two different orthogroups
            if len(o_ec_nan) > 1:
                
                o_ec.loc[ec1_row, o_ec.isnull().any(axis=0)] = o_ec_nan.iloc[0].combine_first(o_ec_nan.iloc[1])
            # If all of them in one row
            elif len(o_ec_nan) == 1:
                o_ec.loc[ec1_row, o_ec.isnull().any(axis=0)] = o_ec.loc[~o_ec.isnull().any(axis=1), o_ec.isnull().any(axis=0)].squeeze()
            return o_ec.loc[[ec1_row]]
        # If only EC1.1 found
        elif ec1_row >= 0:
            o.iloc[[ec1_row]].to_csv(path_to_output_tsv, sep="\t")
            return o.iloc[[ec1_row]]
        # If only EC1.2 found
        elif ec2_row >= 0:
            o.iloc[[ec2_row]].to_csv(path_to_output_tsv, sep="\t")
            return o.iloc[[ec2_row]]


`orthogroups_i.tsv` (i from 1 to 7) are output of the Orthofinder for each of 7 groups.

In [365]:
# All dataframes
dfs = []
for i in range(1, 8):  
    dfs.append(find_ec_orthogroup(f"orthogroups_{i}.tsv", f"res_{i}.tsv").drop(["Orthogroup"], axis=1))

<ipython-input-364-9022e7025c52>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  o_ec.loc[ec1_row, ~o_ec.isnull().any(axis=0)] = o_ec.loc[ec1_row, ~o_ec.isnull().any(axis=0)] + ", " + o_ec.loc[ec2_row, ~o_ec.isnull().any(axis=0)]


In [366]:
# We unite all the species orthologues in one-row table based on Arabidopsis thaliana genes
matching_ids = reduce(lambda x, y: pd.merge(x, y, on = "Arabidopsis_thaliana.TAIR10.pep.all"), dfs)
# Select all columns without nan
matching_ids = matching_ids[matching_ids.columns[~matching_ids.isnull().all()]]

In [367]:
# Flattening the list with all genes
genes = [gene.strip() for sublist in list(matching_ids.iloc[0].str.split(",").values) for gene in sublist]
genes

['PSS26677',
 'AT1G76750.1',
 'AT2G21740.1',
 'AT2G21750.1',
 'AT4G39340.1',
 'KZN08747',
 'KZN08752',
 'KJB57150',
 'KJB60013',
 'KJB71292',
 'Kaladp0026s0134.1.v1.1.CDS.1',
 'Kaladp0033s0187.1.v1.1.CDS.1',
 'Kaladp0040s0523.1.v1.1.CDS.1',
 'Kaladp0515s0048.1.v1.1',
 'Ma06_p27500.1',
 'Ma08_p15310.1',
 'OIT31467',
 'Os03t0296600-01',
 'Os11t0168000-01',
 'ESW18393',
 'ESW28268',
 'ESW31795',
 'PNT12323',
 'PNT20170',
 'PNT46426',
 'PNT57550',
 'PNT57551',
 'PNT58448',
 'ONH89505',
 'ONH90042',
 'ONH90046',
 'ONH90087',
 'ONH90092',
 'ONH90434',
 'ONI01825',
 'ERN08734',
 'Aco003193.1.mrna1.cds1',
 'Aco010349.1.mrna1',
 'Aco016990.1.mrna1.cds1',
 'Aco018685.1.mrna1.cds1',
 'Aco019928.1.mrna1.cds1',
 'Aco024757.1.mrna1.cds1',
 'Aco024977.1.mrna1.cds1',
 'Aco029478.1.mrna1.cds1',
 'Aco031263.1.mrna1.cds1',
 'cds.evm.model.01.1369',
 'cds.evm.model.04.1284',
 'CDP12837',
 'KGN46138',
 'KGN64837',
 'KGN64838',
 'KRG91676',
 'KRG98863',
 'KRH35123',
 'KRH39377',
 'KRH43377',
 'KRH54635',
 '

How many genes found?

In [368]:
len(genes)

206

Here we extract the genes' FASTA amino acid sequences and write them into file `conc_protein_seq.fa`. For this purpose we have all of the `.pep.all.fa` sequences in the directory `protein/`. 

In [369]:
!ls protein/

Actinidia_chinensis.Red5_PS1_1.69.0.pep.all.fa
Amborella_trichopoda.AMTR1.0.pep.all.fa
Ananas_comosus.F153.pep.all.fa
Arabidopsis_thaliana.TAIR10.pep.all.fa
Bdistachyon_314_v3.1.protein.fa
Beta_vulgaris.RefBeet-1.2.2.pep.all.fa
Brassica_oleracea.BOL.pep.all.fa
Camelina_sativa.Cs.pep.all.fa
Cannabis_sativa_female.cs10.pep.all.fa
Citrus_clementina.Citrus_clementina_v1.0.pep.all.fa
Coffea_canephora.AUK_PRJEB4211_v1.pep.all.fa
Cucumis_sativus.ASM407v2.pep.all.fa
Daucus_carota.ASM162521v1.pep.all.fa
Dioscorea_rotundata.TDr96_F1_v2_PseudoChromosome.pep.all.fa
Eucalyptus_grandis.Egrandis1_0.pep.all.fa
Eutrema_salsugineum.Eutsalg1_0.pep.all.fa
Fagopyrum_esculentum.pep.fa
Fagopyrum_tataricum.pep.fa
Ficus_carica.UNIPI_FiCari_1.0.pep.all.fa
Fvesca_677_v4.0.a2.protein.fa
Glycine_max.Glycine_max_v2.1.pep.all.fa
Gossypium_raimondii.Graimondii2_0_v6.pep.all.fa
Helianthus_annuus.HanXRQr2.0-SUNRISE.pep.all.fa
Hordeum_vulgare.IBSC_v2.pep.all.fa
Ipomoea_triloba.ASM357664v1.pep.all.fa
Juglans_regia.Walnut

In [370]:
# IDs of genes
ids = []
# Species: according genes
species_genes_dic = {}

path_to_AA_FASTA_directory = "protein/"
path_to_output_file = "conc_protein_seq.fa"

# Delete the file if it already exists
if os.path.exists(path_to_output_file):
    os.remove(path_to_output_file)

for file in os.listdir("protein"):
    with open(path_to_AA_FASTA_directory + file) as in_file, open(path_to_output_file, "a") as out_file:
        for record in SeqIO.parse(in_file, "fasta"):
            # If record ID in genes, then write it into FASTA file
            if record.id in genes:
                # Extracting gene names, in case explicitly stated
                gene_split = record.description.split("gene:")
                # If gene: stated
                if len(gene_split) > 1:
                    species = file.split(".")[0]
                    species_genes_dic[species] = species_genes_dic.get(species, []) + [gene_split[1].split(" ")[0]]
                # Else - gene name most probably the record id
                else:
                    species = file.split(".")[0]
                    species_genes_dic[species] = species_genes_dic.get(species, []) + [record.id]
                record.id += "_" + file.split(".")[0]
                ids.append(record.id)
                SeqIO.write(record, out_file, "fasta")
            # Where : was changed to _ by OrthoFinder
            elif file.startswith(("Helianthus", "Nymphae", "Ficus", "Quercus", "Malus")):
                record.id = record.id.replace(":", "_")
                if record.id in genes:
                    gene_split = record.description.split("gene:")
                    if len(gene_split) > 1:
                        species = file.split(".")[0]
                        species_genes_dic[species] = species_genes_dic.get(species, []) + [gene_split[1].split(" ")[0]]
                    else:
                        species = file.split(".")[0]
                        species_genes_dic[species] = species_genes_dic.get(species, []) + [record.id]
                    record.id += "_" + file.split(".")[0]
                    ids.append(record.id)
                    SeqIO.write(record, out_file, "fasta")

In [371]:
species_genes_ids = list(itertools.chain.from_iterable(list(species_genes_dic.values())))

In [372]:
len(species_genes_ids)

201

In [373]:
len(ids)

201

Here we have 5 less genes, because the *Hordeum vulgare* Ensembl plants release 53 MorexV3 pseudomolecules assembly version amino acid sequence is not included, as its gene IDs could not be converted to transcriptome IDs. Instead, we use the release 52 genome, IBSC v2. 

Here, we add a missing gene from *Medicago truncatula*, which was discovered when having different 6th and 7th repeat groups, so previous OrthoFinder results. 

In [374]:
species_genes_dic['Medicago_truncatula'] += ["MTR_3g095330"]
species_genes_dic['Medicago_truncatula']

['MTR_8g099500', 'MTR_3g049180', 'MTR_3g049320', 'MTR_3g095330']

Showing file contents, in case names don't match:

In [375]:
!ls annotation/

Actinidia_chinensis.Red5_PS1_1.69.0.52.gff3
Amborella_trichopoda.AMTR1.0.52.gff3
Arabidopsis_thaliana.TAIR10.52.gff3
Bdistachyon_314_v3.1.gene.gff3
Brassica_oleracea.BOL.52.gff3
Camelina_sativa.Cs.52.gff3
Cannabis_sativa_female.cs10.52.gff3
Fvesca_677_v4.0.a2.gene.gff3
Glycine_max.Glycine_max_v2.1.52.gff3
Hordeum_vulgare.IBSC_v2.51.gff3
Mdomestica_491_v1.1.gene.gff3
Medicago_truncatula.MedtrA17_4.0.52.gff3
Nicotiana_attenuata.NIATTr2.52.gff3
Oryza_sativa.IRGSP-1.0.52.gff3
Phalaenopsis_equestris.gff3
Populus_trichocarpa.Pop_tri_v3.52.gff3
Solanum_lycopersicum.SL3.0.52.gff3
Solanum_tuberosum_rh8903916.ASM1418947v1.52.gff3
Sorghum_bicolor.Sorghum_bicolor_NCBIv3.52.gff3
Triticum_aestivum.IWGSC.52.gff3
Vitis_vinifera.12X.52.gff3
Zea_mays.Zm-B73-REFERENCE-NAM-5.0.52.gff3


In [376]:
!ls genome/ 

Actinidia_chinensis.Red5_PS1_1.69.0.dna_sm.toplevel.fa
Amborella_trichopoda.AMTR1.0.dna_sm.toplevel.fa
Arabidopsis_thaliana.TAIR10.dna_sm.toplevel.fa
Bdistachyon_314_v3.0.softmasked.fa
Brassica_oleracea.BOL.dna_sm.toplevel.fa
Camelina_sativa.Cs.dna_sm.toplevel.fa
Cannabis_sativa_female.cs10.dna_sm.toplevel.fa
Fvesca_677_v4.0.a1.softmasked.fa
Glycine_max.Glycine_max_v2.1.dna_sm.toplevel.fa
Hordeum_vulgare.IBSC_v2.dna_sm.toplevel.fa
Mdomestica_491_v1.1.fa
Medicago_truncatula.MedtrA17_4.0.dna_sm.toplevel.fa
Nicotiana_attenuata.NIATTr2.dna_sm.toplevel.fa
Oryza_sativa.IRGSP-1.0.dna_sm.toplevel.fa
Phalaenopsis_equestris.fa
Populus_trichocarpa.Pop_tri_v3.dna_sm.toplevel.fa
Solanum_lycopersicum.SL3.0.dna_sm.toplevel.fa
Solanum_tuberosum_rh8903916.ASM1418947v1.dna_sm.toplevel.fa
Sorghum_bicolor.Sorghum_bicolor_NCBIv3.dna_sm.toplevel.fa
Triticum_aestivum.IWGSC.dna_sm.toplevel.fa
Vitis_vinifera.12X.dna_sm.toplevel.fa
Zea_mays.Zm-B73-REFERENCE-NAM-5.0.dna_sm.toplevel.fa


In [377]:
def extract_annotation(line):
    """
    Extracts contig id, start, end and strand direction of the gene from the GFF3 annotation line
    :param line: str, tab-separated GFF3 annotation line
    :return: list, of contig ID, start, end and strand direction
    """
    line_components = line.split("\t")
    contig_id = line_components[0]
    start = int(line_components[3])
    end = int(line_components[4])
    strand_direction = line_components[6]
    return [contig_id, start, end, strand_direction]

In [378]:
path_to_annotation_directory = "annotation/"
genes_annot = {}

for annotation_file in os.listdir(path_to_annotation_directory):
    with open(os.path.join(path_to_annotation_directory, annotation_file), "r") as annotation:
        for line in annotation:
            for gene in species_genes_dic[annotation_file.split(".")[0]]:
                gene = gene.strip()
                if re.search(gene, line):
                    # In some cases - there are several mrnas per gene, transcriptome data only for one gene anyway...
                    if re.search("\tgene\t", line):
                        genes_annot[gene] = extract_annotation(line)
                    # Sometimes no genes, just mRNAs
                    elif re.search("\tmRNA\t", line) and gene not in genes_annot.keys():
                        genes_annot[gene] = extract_annotation(line)

In [379]:
genes_annot

{'AT1G76750': ['1', 28811040, 28811721, '+'],
 'AT2G21740': ['2', 9281986, 9282387, '-'],
 'AT2G21750': ['2', 9283367, 9283774, '-'],
 'AT4G39340': ['4', 18293129, 18293512, '-'],
 'CEY00_Acc08092': ['LG7', 10802354, 10802982, '+'],
 'AMTR_s00017p00240840': ['AmTr_v1.0_scaffold00017', 5583736, 5584116, '-'],
 'evm.TU.01.1369': ['1', 28222687, 28223799, '+'],
 'evm.TU.04.1284': ['4', 55910526, 55911011, '-'],
 'Csa10g002090': ['10', 519756, 520130, '-'],
 'Csa11g002350': ['11', 625946, 626465, '-'],
 'Csa12g002210': ['12', 549257, 549631, '-'],
 'Csa16g040920': ['16', 20922943, 20923398, '+'],
 'Csa16g043520': ['16', 22265372, 22265749, '-'],
 'Csa16g043530': ['16', 22269739, 22270303, '-'],
 'Csa07g048320': ['7', 24873026, 24873481, '+'],
 'Csa07g048380': ['7', 24897835, 24897996, '-'],
 'Csa07g051880': ['7', 26379055, 26380122, '-'],
 'Csa07g051890': ['7', 26380818, 26381239, '-'],
 'Csa09g081500': ['9', 30963380, 30963838, '+'],
 'Csa09g086140': ['9', 32294595, 32294972, '-'],
 'Csa0

Here, we rename *Malus domestica* genome file, so that it corresponds to species name.

In [129]:
!mv genome/Mdomestica_491_GDDH13v1.1.fa genome/Mdomestica_491_v1.1.fa

Now we save 500 bp nucleotide upstream sequences. 

In [380]:
upstream_length = 500
           
genome_input_directory = "genome/"        
output_file = "upstream_sequences.fa"

if os.path.exists(output_file):
    os.remove(output_file)

# Going over each FASTA file
for fasta_file in os.listdir(genome_input_directory):
    with open(os.path.join(genome_input_directory, fasta_file), "r") as fasta_input, open(output_file, "a") as fasta_output:
        for record in SeqIO.parse(fasta_input, "fasta"):
            for gene in species_genes_dic[fasta_file.split(".")[0]]:
                gene = gene.strip()
                # Saving the variable names
                contig_id = genes_annot[gene][0]
                start = genes_annot[gene][1]
                end = genes_annot[gene][2]   
                strand_direction = genes_annot[gene][3]
                # Checking the strand direction
                if strand_direction == "+":
                    # Choosing the right contig
                    if record.id == contig_id:
                        # Writing output
                        fasta_output.write(">" + gene + "\n" + str(record.seq[start-upstream_length-1:start-1]) + "\n")
                elif strand_direction == "-":
                    if record.id == contig_id:
                        fasta_output.write(">" + gene + "\n" + str(record.seq[end:end + upstream_length].reverse_complement()) + "\n")


Camelina_sativa.Cs.dna_sm.toplevel.fa
Populus_trichocarpa.Pop_tri_v3.dna_sm.toplevel.fa
Mdomestica_491_v1.1.fa
Zea_mays.Zm-B73-REFERENCE-NAM-5.0.dna_sm.toplevel.fa
Triticum_aestivum.IWGSC.dna_sm.toplevel.fa
Solanum_lycopersicum.SL3.0.dna_sm.toplevel.fa
Brassica_oleracea.BOL.dna_sm.toplevel.fa
Vitis_vinifera.12X.dna_sm.toplevel.fa
Hordeum_vulgare.IBSC_v2.dna_sm.toplevel.fa
Amborella_trichopoda.AMTR1.0.dna_sm.toplevel.fa
Phalaenopsis_equestris.fa
Arabidopsis_thaliana.TAIR10.dna_sm.toplevel.fa
Solanum_tuberosum_rh8903916.ASM1418947v1.dna_sm.toplevel.fa
Oryza_sativa.IRGSP-1.0.dna_sm.toplevel.fa
Actinidia_chinensis.Red5_PS1_1.69.0.dna_sm.toplevel.fa
Nicotiana_attenuata.NIATTr2.dna_sm.toplevel.fa
Medicago_truncatula.MedtrA17_4.0.dna_sm.toplevel.fa
Fvesca_677_v4.0.a1.softmasked.fa
Glycine_max.Glycine_max_v2.1.dna_sm.toplevel.fa
Cannabis_sativa_female.cs10.dna_sm.toplevel.fa
Bdistachyon_314_v3.0.softmasked.fa
Sorghum_bicolor.Sorghum_bicolor_NCBIv3.dna_sm.toplevel.fa


Finally, we have the upstream sequences. We divide the sequences into 3 groups, and three separate FASTA files:
1. Generative
2. Non-specific
3. Vegetative

See folder `../Searching_motifs` for further analysis.